# This notebook outlines pre-processing of Amazong Reviews

Most of the work is done in clases that I wrote, but I'll outline what I did for pre-processing in this notebook. Running pre-processing in the notebook took a lot longer so running it via command line seems to be quicker so I can iterate

Add code is checked into my github repo: https://github.com/sv650s/sb-capstone


* preprocess_amazon.py - python program that calls TextProcessor with parameters set to handle the amazon review file
* TextProcessor.py - processor class that uses various utilities to pre-process the file
* text_util.py - has a bunch of text processing methods to clean the data
* file_util.py - functiont to handle files (ie, covert tsv to csv)
* df_util.py - utility to handle pandas DataFrames


Unit Tests:
* TestTextUtil.py - tests text_util.py

To Be Implemented:
* unit tests for file_util.py
* unit tests for pd_util.py


## Before we pre-process, I had to convert tsv to CSV because Pandas was not reading the columns correctly and was putting multiple rows into a column resulting in headline columns that had over 30k words

Original Amazon file had 9mil reviews. I added sampling parameter to reduce the size of the file. Currently, the sampling is pretty dumb. It just grabs every nth line in the file and put it in the final csv file. Will probably rewrite this so it's based on random.rand later

I already ran this via command line because it was faster. So the commented out lines are how we would generate the other sizes.

In [1]:
from file_util import convert_tsv_to_csv

# only need to run this one time
CONVERT_FILE=False

# full 9mil Wireless reviews - not enough memory locally to do this
ORIG_FILE_WIRELESS="dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00.tsv"

# about 22k reviews
DATA_FILE_TEST = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-test.csv"
# about 100023 reviews
DATA_FILE_TINY = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-tiny.csv"
# about 300068 reviews
DATA_FILE_SMALL = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-small.csv"
# about 450101 reviews
DATA_FILE_MEDIUM = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-medium.csv"
# about 900203 reviews
DATA_FILE_LARGE = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-large.csv"

# already ran this
if CONVERT_FILE:
    convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_TEST, 400)
    # convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_TINY, 90)
    # convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_SMALL, 25)
    # convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_MEDIUM, 20)
    # convert_tsv_to_csv(ORIG_FILE_WIRELESS, DATA_FILE_LARGE, 10)


## sample run and output of preprocess_amazon.py

I'm using a file that has only 25k entires so I can run it in the notebook quickly

Notice final output is 19889 because after steming and removing stop words some of the review headlines are now blank

Pre-processing entails the following (in order):
* make everything lowercase
* remove newlines
* remove amazon tags - amazon embeds these [[VIDDEO:dsfljlsjf]] and [[ASIN:sdfjlsjdfl]] tags that need to be removed
* remove html tags - line breaks, etc are represented in reviews as HTML tags
* remove accent characters
* expand contractions - expands contractions like he's but needs to be done before special charaters because we want to expand don't into do not for our text processing
* remove special characters - anything that is not alphanumeric or spaces
* remove stop words - see text_util.py for stop words that I removed from nltk stop words because I think they will be important for sentiment analysis
* stem or lemmatize words - ONLY Porter stemming is implemented currently (THIS IS TURNED OFF)

Columns that it drops right off the bat: marketplace, vine, verified_purchase
Columns that it is pre-processing: product_title, review_headline, review_body

Also, for convenience, there is a flag to retain the original column so we can see the orignal text next to the pre-processed text so we can look for errors. Will not be using this flag for final data files


Here is the list of default stop words from nltk:


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [2]:
!python preprocess_amazon.py -l INFO -r \
    -o dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-testout.csv \
    dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-testin.csv


Traceback (most recent call last):
  File "preprocess_amazon.py", line 84, in <module>
    main()
  File "preprocess_amazon.py", line 58, in main
    logging.setlevel
AttributeError: module 'logging' has no attribute 'setlevel'


In [3]:
import pandas as pd

PREPROCESSED_CSV = "dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-testout.csv"


## Reading the output file back in to look at some data

In [4]:
review_df = pd.read_csv(PREPROCESSED_CSV, parse_dates=["review_date"])
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22410 entries, 0 to 22409
Data columns (total 15 columns):
customer_id             22410 non-null int64
review_id               22410 non-null object
product_id              22410 non-null object
product_parent          22410 non-null int64
product_title_orig      22410 non-null object
product_title           22410 non-null object
product_category        22410 non-null object
star_rating             22410 non-null int64
helpful_votes           22410 non-null int64
total_votes             22410 non-null int64
review_headline_orig    22410 non-null object
review_headline         22410 non-null object
review_body_orig        22410 non-null object
review_body             22410 non-null object
review_date             22410 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(9)
memory usage: 2.6+ MB


In [5]:
# let's sample the dataframe so we can look at some of the data
sample_df = review_df.sample(n=10)

## Product Title

In [6]:
df = review_df
df["product_title_orig_len"] = df["product_title_orig"].apply(lambda x: len(x))
df["product_title_len"] = df["product_title"].apply(lambda x: len(x))

df["product_title_diff"] = df["product_title_len"] - df["product_title_orig_len"]
df["product_title_percent_diff"] = df["product_title_diff"] / df["product_title_orig_len"]



df[["product_title_orig_len", "product_title_len", "product_title_diff", "product_title_percent_diff"]].describe()

,product_title_orig_len,product_title_len,product_title_diff,product_title_percent_diff
count,22410.000000,22410.000000,22410.000000,22410.000000
mean,104.419277,94.322445,-10.096832,-0.088215
std,61.652263,54.613601,9.132490,0.053863
min,3.000000,3.000000,-86.000000,-0.470588
25%,60.000000,55.000000,-14.000000,-0.120879
50%,90.000000,82.000000,-8.000000,-0.086957
75%,136.000000,122.000000,-4.000000,-0.053191
max,400.000000,400.000000,0.000000,0.000000


In [7]:
import numpy as np

rows = len(sample_df)
# first let's randomly look at a couple rows
for i in np.arange(0,3,1):
    row_index = round(np.random.rand() * rows - 1)
    row = sample_df.iloc[row_index]
    print(f'product_title_orig\t[{row["product_title_orig"]}]')
    print(f'product_title\t\t[{row["product_title"]}]')




product_title_orig	[Cellet 4" Bike & Motorcycle Cell Phone Holder Mount for Nokia Lumia 1520]
product_title		[cellet 4 bike motorcycle cell phone holder mount nokia lumia 1520]
product_title_orig	[Faber-Castell Duo Tip Washable]
product_title		[faber castell duo tip washable]
product_title_orig	[Faber-Castell Duo Tip Washable]
product_title		[faber castell duo tip washable]


## I did notice that in product titles - things like 5.5 get converted to 5 5 - will have to do something about this if we decide to use product title

## review headlines

average 22% reduction in length with stemming turned on

average 17% reduction in length with stemming turned off

In [8]:

df = review_df
df["review_headline_orig_len"] = df["review_headline_orig"].apply(lambda x: len(x))
df["review_headline_len"] = df["review_headline"].apply(lambda x: len(x))

df["review_headline_diff"] = df["review_headline_len"] - df["review_headline_orig_len"]
df["review_headline_percent_diff"] = df["review_headline_diff"] / df["review_headline_orig_len"]

df[["review_headline_orig_len", "review_headline_len", "review_headline_diff", "review_headline_percent_diff"]].describe()

,review_headline_orig_len,review_headline_len,review_headline_diff,review_headline_percent_diff
count,22410.000000,22410.000000,22410.000000,22410.000000
mean,22.041187,16.727755,-5.313432,-0.168167
std,18.062227,12.014138,8.247019,0.199324
min,1.000000,1.000000,-68.000000,-0.904762
25%,10.000000,10.000000,-8.000000,-0.309524
50%,15.000000,12.000000,-1.000000,-0.083333
75%,28.000000,21.000000,0.000000,0.000000
max,128.000000,125.000000,0.000000,0.000000


In [9]:
# let's now look at review_headline
# first let's randomly look at a couple rows
for i in np.arange(0,5,1):
    row_index = round(np.random.rand() * rows -1)
    row = sample_df.iloc[row_index]
    print(f'review_headline_orig\t[{row["review_headline_orig"]}]')
    print(f'review_headline\t\t[{row["review_headline"]}]')

review_headline_orig	[Cheap]
review_headline		[cheap]
review_headline_orig	[BEST OVER THE EAR HEADPHONES... GREAT SOUND AND COMFORTABLE]
review_headline		[best ear headphones great sound comfortable]
review_headline_orig	[OtterBox case & Holster]
review_headline		[otterbox case holster]
review_headline_orig	[Slim Fit Clear Case]
review_headline		[slim fit clear case]
review_headline_orig	[Four Stars]
review_headline		[four stars]


## review body - pretty significant reduction in length

Average 40% reduction in size of review body if stemming is turned on

Once you turn off stemming, about 35% average reduction in sbody size

In [10]:
df = review_df
df["review_body_orig_len"] = df["review_body_orig"].apply(lambda x: len(x))
df["review_body_len"] = df["review_body"].apply(lambda x: len(x))
df["review_body_diff"] = df["review_body_len"] - df["review_body_orig_len"]
df["review_body_percent_diff"] = df["review_body_diff"] / df["review_body_orig_len"]
df[["review_body_orig_len", "review_body_len", "review_body_diff", "review_body_percent_diff"]].describe()

,review_body_orig_len,review_body_len,review_body_diff,review_body_percent_diff
count,22410.000000,22410.000000,22410.000000,22410.000000
mean,256.354440,158.150959,-98.203481,-0.345040
std,420.598432,258.101204,164.786304,0.128854
min,1.000000,1.000000,-3426.000000,-0.891892
25%,70.000000,45.000000,-109.000000,-0.423313
50%,141.000000,89.000000,-53.000000,-0.368421
75%,281.000000,171.000000,-22.000000,-0.295858
max,9888.000000,6519.000000,0.000000,0.000000


In [11]:
# let's now look at review_body
# first let's randomly look at a couple rows
for i in np.arange(0,10,1):
    row_index = round(np.random.rand() * rows - 1)
    row = sample_df.iloc[row_index]
    print(f'review_body_orig\t[{row["review_body_orig"]}]')
    print(f'review_body\t\t[{row["review_body"]}]\n')

review_body_orig	[This is my 2nd pair of &#34;over the ear&#34; bluetooth headphones. I started back at the gym 6 months ago and used the typical ear buds with a cord. It didn't take long before I was losing my phone on the treadmill because of the cord catching a part of my body. I am huge on quality of sound and how well I can keep sound out. I bought a set of JAMS first and they lasted me for about 4 months. I absolutely loved them but the left side completely went out. I read reviews on this problem shortly after it happened. I really wanted that great sound (bass in particular) and was skeptical to buy another set of JAMS. After research and reviews, I found these bad boys. They are a little bulky but hey, so are the BEATS. Whats better than the BEATS is that they are only $50!!! I have never tried BEATS ($299) but I can tell you, I don't know how you could possibly get any better sound...<br />I use these for mowing the yard and the gym. That said, I do a serious amount of sweati